<h1>Machine learning using Neural networks to predict eye state data</h1>

In [17]:
# import libraries
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from matplotlib import pyplot

In [9]:
# Load in the data frame
raw_data = loadarff('data/EEG Eye State.arff')
df_data = pd.DataFrame(raw_data[0])
print(df_data)

           AF3       F7       F3      FC5       T7       P7       O1       O2  \
0      4329.23  4009.23  4289.23  4148.21  4350.26  4586.15  4096.92  4641.03   
1      4324.62  4004.62  4293.85  4148.72  4342.05  4586.67  4097.44  4638.97   
2      4327.69  4006.67  4295.38  4156.41  4336.92  4583.59  4096.92  4630.26   
3      4328.72  4011.79  4296.41  4155.90  4343.59  4582.56  4097.44  4630.77   
4      4326.15  4011.79  4292.31  4151.28  4347.69  4586.67  4095.90  4627.69   
...        ...      ...      ...      ...      ...      ...      ...      ...   
14975  4281.03  3990.26  4245.64  4116.92  4333.85  4614.36  4074.87  4625.64   
14976  4276.92  3991.79  4245.13  4110.77  4332.82  4615.38  4073.33  4621.54   
14977  4277.44  3990.77  4246.67  4113.85  4333.33  4615.38  4072.82  4623.59   
14978  4284.62  3991.79  4251.28  4122.05  4334.36  4616.41  4080.51  4628.72   
14979  4287.69  3997.44  4260.00  4121.03  4333.33  4616.41  4088.72  4638.46   

            P8       T8    

In [68]:
# Preprocess Data
data_targets=df_data.iloc[:,-1]
data=df_data.iloc[:,:-1]

s = pd.Series(data_targets)
data_targets=pd.get_dummies(s)
data_targets=data_targets.iloc[:,1]

In [70]:
# split the data
xTrain, xTest, yTrain, yTest = train_test_split(data, data_targets, test_size = 20, random_state = 25)

In [71]:
# train the neural net
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(14, 10, 4, 2), random_state=1)
clf.fit(xTrain, yTrain)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(14, 10, 4, 2), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [74]:
# evaluate model
clf.predict(xTest)